In [552]:
from matplotlib import pyplot as plt
import numpy as np
import maxpy as mp
import time

from pythonosc.udp_client import SimpleUDPClient
from pythonosc.dispatcher import Dispatcher
from pythonosc.osc_server import BlockingOSCUDPServer

from pyfirmata2 import Arduino 

In [438]:
np.random.random(1)

array([0.66354865])

In [448]:
def rand_range(min, max):
    rand = float(np.random.random(1)[0])

    range = max - min
    shifted = rand * range + min
    return shifted
    

In [554]:
for i in range(2, 10):
    print (i)

2
3
4
5
6
7
8
9


In [570]:
class Phonebook:

    ip = "127.0.0.1"
    port = 8000

    norecall = 0.7
    recall = 0.3

    def __init__(self, n, hl, pl): 

        self.lines = []
        self.num_lines = n
        self.not_busy = [x for x in range(n)] #keep track of busy lines 
        self.busy = []
        self.waiting = []
        self.sleepy = []

        self.hl = hl
        self.pl = pl
        
        self.client = SimpleUDPClient(ip, port)

        for i in range(n):
            self.lines.append(self.Telephone(self, i))

        self.reset()

        return


    def reset(self):

        for i in range(self.num_lines):
            flag1 = "/" + str(i) + "-" + "home"
            flag2 = "/" + str(i) + "-" + "phone"
            flag3 = "/" + str(i) + "-" + "msg"
            flag4 = "/" + str(i) + "-" + "wait"
            flag5 = "/" + str(i) + "-" + "dial"
            flag6 = "/" + str(i) + "-" + "receive"
            flag7 = "/" + str(i) + "-" + "talk"
            self.client.send_message(flag1, 0)
            self.client.send_message(flag2, 0)
            self.client.send_message(flag3, " ")
            self.client.send_message(flag4, 0)
            self.client.send_message(flag5, 0)
            self.client.send_message(flag6, 0)
            self.client.send_message(flag7, 0)

            for i in range(2, 12): #homelights through 2 - 11
                self.hl.digital[i].write(False)

            #phonelights 2-10, 12
            for i in range(2, 13):
                self.pl.digital[i].write(False)
                
            
        return


    def report(self, num, loc, msg):

        flag = "/" + str(num) + "-" + loc
        self.client.send_message(flag, msg)

        if loc == "home":
            if msg == 1:
                self.hl.digital[num+2].write(True)
            elif msg == 0:
                self.hl.digital[num+2].write(False)

        elif loc == "phone":
            if num == 9:
                num += 1

            if msg == 1:
                self.pl.digital[num+2].write(True)
            elif msg == 0:
                self.pl.digital[num+2].write(False)
        

        return

    def __repr__(self):

        string = ""

        for i in range(self.num_lines):
            string += str(self.lines[i]) + "\n"

        return string

    def __str__(self):

        string = ""

        for i in range(self.num_lines):
            string += str(self.lines[i]) + "\n"

        return string

    
    class Telephone: 

        def __init__(self, phonebook, num): 

            self.number = num
            self.phonebook = phonebook
            self.busy = False
            self.calling = False
            self.receiving = False
            self.talking = False
            self.waiting = False
            self.awake = False
            self.sleepy = False
            
            self.friend = None

            #time in seconds
            self.dial_time = np.inf #how long for caller to wait before hangup 
            self.pickup_time = np.inf #how long for receiver to wait before pickup
            self.hangup_time = np.inf #how long for receiver to wait before hangup
            self.talk_time = np.inf #how long to talk before hangup 
            self.wait_time = np.inf #how long to wait before calling someone else
            
            self.zero_time = None #global time at which to start counting

            self.will_pickup = False #whether or not receiver will pick up 

            self.will_call = False #whether or not phone will call someone else
            
            return


        def wakeup(self):
            self.awake = True
            self.phonebook.report(self.number, "home", 1)
            return

        def sleep(self):
            self.awake = False
            self.phonebook.report(self.number, "home", 0)
            return

        def set_dialtime(self):
            self.dial_time = rand_range(6, 10)
            print(self.number, "dialtime", self.dial_time)
            return

        def choose_pickup(self):

            self.will_pickup = np.random.choice([0, 1], p=[0.2, 0.8])
            print(self.number, "will pickup?", self.will_pickup)
            if self.will_pickup:
                self.set_pickuptime()
            else:
                self.set_hanguptime()
            return

        def set_pickuptime(self):
            self.pickup_time = rand_range(3, 7)
            print(self.number, "pickuptime", self.pickup_time)
            return

        def set_hanguptime(self):
            self.hangup_time = rand_range(3, 7)
            print(self.number, "hanguptime", self.hangup_time)
            return

        def set_talktime(self):
            self.talk_time = np.random.randint(5, 10)
            print(self.number, "talktime", self.talk_time)
            return

        def set_willcall(self, global_time):
            self.will_call = np.random.choice([0, 1], p=[Phonebook.norecall, Phonebook.recall])
            print(self.number, "will recall?", self.will_call)
            if self.will_call:
                self.set_waittime(global_time)
            else:
                self.sleepy = True
                self.phonebook.sleepy.append(self.number)
                self.zero_time = global_time
            return

        def set_waittime(self, global_time):
            
            self.wait_time = rand_range(3, 7)
            self.zero_time = global_time
            self.waiting = True
            self.sleepy = False
            if self.number in self.phonebook.sleepy:
                self.phonebook.sleepy.remove(self.number)
            self.phonebook.waiting.append(self.number)

            self.phonebook.report(self.number, "wait", 1)
            self.phonebook.report(self.number, "msg", "waiting")
            print(self.number, "waittime", self.wait_time, "zero time", self.zero_time)
            return
        
    
        def call(self, num, global_dt):

            if not self.busy:

                print(str(self.number), "calling", str(num)) 
    
                #flag self + define time
                self.calling = True
                self.busy = True
                self.waiting = False #no longer waiting!
                self.wait_time = np.inf
                if self.number in self.phonebook.waiting:
                    self.phonebook.waiting.remove(self.number)
                self.phonebook.report(self.number, "wait", 0)
                self.friend = num
                self.set_dialtime()
                self.zero_time = global_dt
    
                #flag friend 
                self.phonebook.lines[num].receiving = True
                self.phonebook.lines[num].busy = True
                self.phonebook.lines[num].waiting = False #don't wait to call if ur being called!
                self.phonebook.lines[num].wait_time = np.inf
                if num in self.phonebook.waiting:
                    self.phonebook.waiting.remove(num)
                self.phonebook.lines[num].sleepy = False #can't be sleepy if ur being called
                if num in self.phonebook.sleepy:
                    self.phonebook.sleepy.remove(num)
                self.phonebook.report(num, "wait", 0)
                self.phonebook.lines[num].friend = self.number
                self.phonebook.lines[num].choose_pickup()
                self.phonebook.lines[num].zero_time = global_dt
    
                #flag in phonebook
                self.phonebook.not_busy.remove(num)
                self.phonebook.not_busy.remove(self.number)
                self.phonebook.busy.append(num)
                self.phonebook.busy.append(self.number)
    
                #report to client
                self.phonebook.report(self.number, "phone", 1)
                self.phonebook.report(self.number, "dial", 1)
                self.phonebook.report(self.number, "msg", "calling " + str(self.friend))
                self.phonebook.report(self.friend, "phone", 1)
                self.phonebook.report(self.friend, "receive", 1)
                self.phonebook.report(self.friend, "msg", "receiving " + str(self.number))
            
            return

        def pickup(self, global_time):

            if self.receiving:

                print(str(self.number), "picked up call from ", str(self.friend))

                #flag self 
                self.receiving = False
                self.talking = True
                self.set_talktime()
                self.zero_time = global_time

                #flag friend
                self.phonebook.lines[self.friend].calling = False
                self.phonebook.lines[self.friend].talking = True
                self.phonebook.lines[self.friend].set_talktime()
                self.phonebook.lines[self.friend].zero_time = global_time

                #report to client
                self.phonebook.report(self.number, "pickup", 1)
                self.phonebook.report(self.number, "receive", 0)
                self.phonebook.report(self.number, "talk", 1)
                self.phonebook.report(self.number, "msg", "talking " + str(self.friend))
                self.phonebook.report(self.friend, "dial", 0)
                self.phonebook.report(self.friend, "talk", 1)
                self.phonebook.report(self.friend, "msg", "talking " + str(self.number))

            else:

                if not self.busy:
                    print(str(self.number), "not called, nothing to pickup") 
                else:
                    print("ERROR:", str(self.number), "tried to pick up when busy")

            return 


        def hangup(self, global_time):

            if self.calling:

                #no response
                print(str(self.number), "hanging up on", str(self.friend), ", no response")

                #report to client
                self.phonebook.report(self.number, "msg", " ")
                self.phonebook.report(self.friend, "msg", " ")
                self.phonebook.report(self.number, "phone", 0)
                self.phonebook.report(self.friend, "phone", 0)
                self.phonebook.report(self.number, "dial", 0)
                self.phonebook.report(self.friend, "receive", 0)
                self.phonebook.report(self.number, "hangup", 1)

                #flag friend
                self.phonebook.lines[self.friend].receiving = False
                self.phonebook.lines[self.friend].busy = False
                self.phonebook.lines[self.friend].friend = None
                self.phonebook.lines[self.friend].set_willcall(global_time)

                #flag phonebook
                self.phonebook.not_busy.append(self.friend)
                self.phonebook.not_busy.append(self.number)
                self.phonebook.busy.remove(self.friend)
                self.phonebook.busy.remove(self.number)

                #flag self 
                self.calling = False
                self.busy = False
                self.friend = None
                self.set_willcall(global_time)


            elif self.receiving:

                #don't want to pick up 
                print(str(self.number), "hanging up on", str(self.friend), ", don't want to talk")

                #report to client
                self.phonebook.report(self.number, "msg", " ")
                self.phonebook.report(self.friend, "msg", " ")
                self.phonebook.report(self.number, "phone", 0)
                self.phonebook.report(self.friend, "phone", 0)
                self.phonebook.report(self.number, "receive", 0)
                self.phonebook.report(self.friend, "dial", 0)
                self.phonebook.report(self.number, "hangup", 1)
                
                #flag friend
                self.phonebook.lines[self.friend].calling = False
                self.phonebook.lines[self.friend].busy = False
                self.phonebook.lines[self.friend].friend = None
                self.phonebook.lines[self.friend].set_willcall(global_time)

                #flag phonebook
                self.phonebook.not_busy.append(self.friend)
                self.phonebook.not_busy.append(self.number)
                self.phonebook.busy.remove(self.friend)
                self.phonebook.busy.remove(self.number)

                #flag self 
                self.receiving = False
                self.busy = False
                self.friend = None
                self.set_willcall(global_time)

            elif self.talking:

                #done talking
                print(str(self.number), "hanging up on", str(self.friend), ", done talking")

                #report to client
                self.phonebook.report(self.number, "msg", " ")
                self.phonebook.report(self.friend, "msg", " ")
                self.phonebook.report(self.number, "phone", 0)
                self.phonebook.report(self.friend, "phone", 0)
                self.phonebook.report(self.number, "talk", 0)
                self.phonebook.report(self.friend, "talk", 0)
                self.phonebook.report(self.number, "hangup", 1)
                self.phonebook.report(self.friend, "hangup", 1)

                #flag friend
                self.phonebook.lines[self.friend].talking = False
                self.phonebook.lines[self.friend].busy = False
                self.phonebook.lines[self.friend].friend = None 
                self.phonebook.lines[self.friend].set_willcall(global_time)

                #flag phonebook
                self.phonebook.not_busy.append(self.friend)
                self.phonebook.not_busy.append(self.number)
                self.phonebook.busy.remove(self.friend)
                self.phonebook.busy.remove(self.number)

                #flag self 
                self.talking = False
                self.busy = False
                self.friend = None
                self.set_willcall(global_time)

            else:

                print("ERROR:", str(self.number), "tried to hang up without calling")

            return



            

        def __repr__(self):
            
            string = "\nPhone number: " + str(self.number)

            if self.busy:
                string += ", BUSY"

            if self.calling:
                string += ", calling " + str(self.friend) + ", dialtime " + str(self.dial_time) + ", talktime " + str(self.talk_time)

            if self.receiving:
                string += ", receiving " + str(self.friend)
                if self.will_pickup:
                    string += ", pickuptime " + str(self.pickup_time) + ", talktime " + str(self.talk_time)
                else:
                    string += ", hanguptime " + str(self.hangup_time)

            if self.talking:
                string += ", talking to " + str(self.friend) + ", talktime " + str(self.talk_time)
            
            return string

        def __str__(self):

            string = "\nPhone number: " + str(self.number)

            if self.busy:
                string += ", BUSY"

            if self.calling:
                string += ", calling " + str(self.friend) + ", dialtime " + str(self.dial_time) + ", talktime " + str(self.talk_time)

            if self.receiving:
                string += ", receiving " + str(self.friend)
                if self.will_pickup:
                    string += ", pickuptime " + str(self.pickup_time) + ", talktime " + str(self.talk_time)
                else:
                    string += ", hanguptime " + str(self.hangup_time)

            if self.talking:
                string += ", talking to " + str(self.friend) + ", talktime " + str(self.talk_time)
            
            return string
    

In [611]:
homelights = Arduino("/dev/cu.usbmodem1101")
phonelights = Arduino("/dev/cu.usbmodem3101")

In [612]:
phonebook = Phonebook(10, homelights, phonelights)
phones = phonebook.lines

In [613]:
t_total = 100
dt = 0.1

num_dt = int(t_total/dt)

In [614]:
#starting caller(s)
starters = [0, 5, 7, 9]
curr_time = 0

#first, turn on lights in calling homes
for i in range(20):

    if i == 0:
        for i in range(len(starters)):
            phone = phones[starters[i]]
            phone.wakeup()

    time.sleep(dt)

for t in range(num_dt):

    if t%10 == 0:
        print(curr_time)

    #start off calling
    if t == 0:
        for i in range(len(starters)):
    
            phone = starters[i]
            not_busy = [x for x in phonebook.not_busy if x not in starters]
            if phone in not_busy:
                not_busy.remove(phone)
            phones[phone].call(np.random.choice(not_busy), curr_time)


    #check busy phones to take action
    busy_phones = []
    for i in range(len(phonebook.busy)):

        phone = phones[phonebook.busy[i]]
        busy_phones.append(phone)

    for phone in busy_phones:
        phonetime = curr_time - phone.zero_time

        if phone.calling and phonetime > phone.dial_time:
            phone.hangup(curr_time)
            

        elif phone.receiving and not phone.will_pickup and not phone.awake and phonetime > phone.hangup_time - 2:
            phone.wakeup()
            
        elif phone.receiving and not phone.will_pickup and phone.awake and phonetime > phone.hangup_time:
            phone.hangup(curr_time)


        elif phone.receiving and phone.will_pickup and not phone.awake and phonetime > phone.pickup_time - 2:
            phone.wakeup()

        elif phone.receiving and phone.will_pickup and phone.awake and phonetime > phone.pickup_time:
            phone.pickup(curr_time)

        elif phone.talking and phonetime > phone.talk_time:
            phone.hangup(curr_time)

    waiting_phones = []
    for i in range(len(phonebook.waiting)):
        phone = phones[phonebook.waiting[i]]
        waiting_phones.append(phone)

    for phone in waiting_phones:
        phonetime = curr_time - phone.zero_time

        if phone.waiting and phonetime > phone.wait_time:
            not_busy = [x for x in phonebook.not_busy]
            if phone.number in not_busy:
                not_busy.remove(phone.number)

            if len(not_busy) != 0:
                phone.call(np.random.choice(not_busy), curr_time)
            else:
                phone.set_waittime(curr_time)

    sleepy_phones = []
    for i in range(len(phonebook.sleepy)):
        phone = phones[phonebook.sleepy[i]]
        sleepy_phones.append(phone)

    for phone in sleepy_phones:
        phonetime = curr_time - phone.zero_time

        if phone.sleepy and phonetime > 2:
            phone.sleep()
    
    curr_time += dt
    time.sleep(dt)

0
0 calling 3
0 dialtime 9.784758188088619
3 will pickup? 1
3 pickuptime 5.751200309023998
5 calling 1
5 dialtime 8.600788769215656
1 will pickup? 0
1 hanguptime 4.96495977509673
7 calling 6
7 dialtime 7.282662682735435
6 will pickup? 0
6 hanguptime 6.942237310888361
9 calling 8
9 dialtime 9.490905438076586
8 will pickup? 1
8 pickuptime 5.1986150600677306
0.9999999999999999
2.0000000000000004
3.0000000000000013
4.000000000000002
4.999999999999998
1 hanging up on 5 , don't want to talk
5 will recall? 0
1 will recall? 0
8 picked up call from  9
8 talktime 6
9 talktime 6
3 picked up call from  0
3 talktime 7
0 talktime 8
5.999999999999995
6.999999999999991
6 hanging up on 7 , don't want to talk
7 will recall? 0
6 will recall? 0
7.999999999999988
8.999999999999984
9.99999999999998
10.999999999999977
8 hanging up on 9 , done talking
9 will recall? 1
9 waittime 5.044458962338023 zero time 11.299999999999976
8 will recall? 0
11.999999999999973
3 hanging up on 0 , done talking
0 will recall? 0

KeyboardInterrupt: 

In [ ]:
np.random.choice([0, 1], p=[0.25, 0.75])